In [1]:
### calculate daily ERA5 wind speed from u- and v-components and place onto Will's 160x160 grid

/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
import numpy as np
import tqdm
from netCDF4 import Dataset
from math import sqrt
from regrid import regrid
import warnings
warnings.filterwarnings("ignore")

In [6]:
# creating numpy are with daily wind data on WG grid for 2010-2020

datapath = '/Users/carmennab/PhD/OI_PolarSnow/ERA5/wind/'
lats = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lat.npy')
lons = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lon.npy')
missing = np.full((153,160,160),np.nan)
years = ['2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

all_v = {}
all_u = {}

for year in tqdm.tqdm(years):
    
    # load ERA5 temperature data
    file = datapath + year +'.nc'
    data = Dataset(file,'r')
    lon = data['longitude']
    lat = data['latitude']
    era5_lon,era5_lat = np.meshgrid(np.array(lon),np.array(lat))
    wind_v = data['v10']
    wind_u = data['u10']   
           
    # turn hourly data into daily data and regrid onto WG (160x160) grid        
    v_daily = []
    u_daily = []
    speed_daily = []

    for i in np.arange(0,wind_v.shape[0],24):

        # v component

        v_on_day = wind_v[i:i+24]

        v_day_ave = np.nanmean(v_on_day,axis=0)
        v_regrid_ave = regrid(v_day_ave.ravel(), era5_lon.ravel(), era5_lat.ravel(), lons, lats,'linear')


        v_daily.append(v_regrid_ave)
        
        # u component

        u_on_day = wind_v[i:i+24]

        u_day_ave = np.nanmean(u_on_day,axis=0)

        u_regrid_ave = regrid(u_day_ave.ravel(), era5_lon.ravel(), era5_lat.ravel(), lons, lats,'linear')

        u_daily.append(u_regrid_ave)        
        
    # create full year list, with NaNs for days in May - September
    year_v = np.concatenate((v_daily[:-92],missing,v_daily[-92:]),axis=0)
    year_u = np.concatenate((u_daily[:-92],missing,u_daily[-92:]),axis=0)

    all_v[year] = year_v
    all_u[year] = year_u

100%|████████████████████████████████████████| 11/11 [2:41:58<00:00, 883.49s/it]


In [7]:
# create one array containing data for all years and save
all_v_arr = np.concatenate((all_v['2010'], all_v['2011'], all_v['2012'], all_v['2013'], 
                               all_v['2014'], all_v['2015'], all_v['2016'], all_v['2017'], 
                               all_v['2018'], all_v['2019'], all_v['2020']),axis=0)
# np.save('/Users/carmennab/PhD/OI_PolarSnow/ERA5/wind/all_v_2010-2020.npy',all_v_arr)
all_u_arr = np.concatenate((all_u['2010'], all_u['2011'], all_u['2012'], all_u['2013'], 
                               all_u['2014'], all_u['2015'], all_u['2016'], all_u['2017'], 
                               all_u['2018'], all_u['2019'], all_u['2020']),axis=0)
# np.save('/Users/carmennab/PhD/OI_PolarSnow/ERA5/wind/all_u_2010-2020.npy',all_u_arr)

In [ ]:
# to skip long calculation above:
all_v_arr = np.load('/Users/carmennab/PhD/OI_PolarSnow/ERA5/wind/all_v_2010-2020.npy')
all_u_arr = np.load('/Users/carmennab/PhD/OI_PolarSnow/ERA5/wind/all_u_2010-2020.npy')

In [ ]:
# calculate daily average wind speed and save array
speed = np.full((len(all_v_arr),160,160),np.nan)
for day in tqdm.tqdm(np.arange(0,len(all_v_arr))):
    for i, j in itertools.product(np.arange(160),
                                  np.arange(160)):
        speed[day,i,j] = sqrt((all_v_arr[day,i,j]**2) + (all_u_arr[day,i,j]**2))
# np.save('/Users/carmennab/PhD/OI_PolarSnow/ERA5/wind/speed_2010-2020.npy',speed)